In [ ]:
import pandas as pd

def load_data(depscore_path, zscore_path):
    """Загружает данные, пропуская строку с процентами."""
    depscore_df = pd.read_csv(depscore_path, index_col=0).iloc[1:]
    zscore_df = pd.read_csv(zscore_path, index_col=0).iloc[1:]
    return depscore_df, zscore_df

def find_genes(depscore_threshold, zscore_threshold, depscore_df, zscore_df):
    """Находит гены, удовлетворяющие заданным порогам depscore и z-score."""
    # Получаем списки всех порогов из заголовков таблиц
    depscore_columns = depscore_df.columns[1:]  # Пропускаем 'Average_Score'
    zscore_columns = zscore_df.columns
    
    # Преобразуем названия столбцов в числовые пороги
    depscore_thresholds = [float(col.split()[-1]) for col in depscore_columns]
    zscore_thresholds = [float(col) for col in zscore_columns]
    
    # Находим столбцы, которые нужно проверить (все <= заданного порога)
    depscore_check_cols = [
        col for col, thr in zip(depscore_columns, depscore_thresholds) 
        if thr <= depscore_threshold
    ]
    zscore_check_cols = [
        col for col, thr in zip(zscore_columns, zscore_thresholds) 
        if thr <= zscore_threshold
    ]
    
    # Если нет подходящих столбцов — возвращаем пустой список
    if not depscore_check_cols or not zscore_check_cols:
        return []
    
    # Находим гены, где в обоих файлах стоит "Да" хотя бы в одном из нужных столбцов
    matching_genes = []
    for gene in depscore_df.index:
        if gene in zscore_df.index:
            depscore_ok = any(depscore_df.loc[gene, col] == "Да" for col in depscore_check_cols)
            zscore_ok = any(zscore_df.loc[gene, col] == "Да" for col in zscore_check_cols)
            if depscore_ok and zscore_ok:
                matching_genes.append(gene)
    
    return matching_genes

def save_genes_to_csv(genes, depscore_threshold, zscore_threshold, output_dir="."):
    """Сохраняет список генов в CSV с названием, отражающим пороги."""
    filename = f"AML_depscore_{depscore_threshold}_zscore_{zscore_threshold}.csv"
    path = f"{output_dir}/{filename}"
    pd.DataFrame(genes, columns=["Gene"]).to_csv(path, index=False)
    print(f"Сохранено в {path}")

# Пример использования
depscore_path = r"../mean_depscores/mean_depscores_by_cancer_type/mean_depscore_aml.csv"
zscore_path = r"../mean_z-scores_by_cancer_type/mean_z-score_aml.csv"

# Загружаем данные
depscore_df, zscore_df = load_data(depscore_path, zscore_path)

# Задаем пороговые значения
depscore_threshold = -0.2  # <= -0.1
zscore_threshold = -1.5   # <= -1.5

# Находим гены
matching_genes = find_genes(depscore_threshold, zscore_threshold, depscore_df, zscore_df)

# Сохраняем в CSV
save_genes_to_csv(matching_genes, depscore_threshold, zscore_threshold)

C:\Users\rusla\AppData\Local\Temp\ipykernel_36068\2711394272.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  depscore_df = pd.read_csv(depscore_path, index_col=0).iloc[1:]


Сохранено в ./AML_depscore_-0.2_zscore_-1.5.csv
